In [ ]:
from google.colab import drive
# drive.mount('/content/gdrive')
drive.mount("/content/gdrive", force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


#Imports

In [ ]:
import math
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
from matplotlib import pyplot
from pandas.plotting import lag_plot, autocorrelation_plot
import numpy as np
from statsmodels.tsa import stattools
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.stattools import acf, pacf
from pandas.core.window import Rolling
from datetime import timedelta, date
%matplotlib inline
import matplotlib
matplotlib.rcParams['figure.figsize'] = [8, 3]
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels
import scipy
from scipy.stats import pearsonr
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf
import itertools
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")
import sklearn as sk
from sklearn import metrics
## utilities
import os
from pandas.core.window import Rolling
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [ ]:
df = pd.read_csv('/content/gdrive/My Drive/crime.csv')

# Data Preperation/Calculations

In [ ]:
#drops unneccesary
df.drop(df.columns[[0, 1, 2, 3, 4, 5, 6, 7]], axis = 1, inplace = True)
df.drop(df.columns[[1, 2, 3]], axis = 1, inplace = True)
df.drop(df.columns[[2, 3]], axis = 1, inplace = True)
#seperates dates and times and makes new columns
new = df["DateTime"].str.split(" ", n = 1, expand = True) 
df["Date"]= new[0] 
df["Time"]= new[1] 
df.drop(df.columns[[2]], axis = 1, inplace = True)
only_CASTROVALLEY = df[df.City.isin(["CASTRO VALLEY", "CASTRO", "CASTRO VALLE", "CDASTRO VALLEY", "CDASTRO VALLEY", "CATRO VALLEY"])]
only_CASTROVALLEY_mental = only_CASTROVALLEY[only_CASTROVALLEY.CrimeDescription.isin(['72 HOUR MENTAL HEALTH'])]
df = only_CASTROVALLEY_mental.groupby(only_CASTROVALLEY_mental.Date).count()
df = df.drop(columns=['CrimeDescription', 'Time'])
df = df.rename(columns={"Date": "Date", "City": "X"})
df.dropna()
idx = pd.date_range('01-01-2012', '06-07-2020')
df.index = pd.DatetimeIndex(df.index)

In [ ]:
df1 = df.reindex(idx, fill_value=0)

In [ ]:
df3 = df.reindex(idx)

In [ ]:
df2 = df3.fillna(df3.rolling(7,min_periods=1).mean())

In [ ]:
df4 = df3.fillna(df3.rolling(7,min_periods=1).median())

#Models

In [ ]:
sarimaErrors = []
def SARIMA(train, test):
    def find_best_sarima(train, eval_metric):
    
        p = d = q = range(0, 2)
        pdq = list(itertools.product(p, d, q))
        seasonal_pdq = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))]

        counter = 0
        myDict = {}

        for param in pdq:
            for param_seasonal in seasonal_pdq:
                try:
                    counter += 1
                    mod = sm.tsa.statespace.SARIMAX(train,
                                                    order=param,
                                                    seasonal_order=param_seasonal,
                                                    enforce_stationarity=False,
                                                    enforce_invertibility=False)

                    results = mod.fit()
                    myDict[counter] = [results.aic, results.bic, param, param_seasonal]

                except:
                    continue

        dict_to_df = pd.DataFrame.from_dict(myDict,orient='index')

        if eval_metric == 'aic':
            best_run = dict_to_df[dict_to_df[0] == dict_to_df[0].min()].index.values
            best_run = best_run[0]
        elif eval_metric == 'bic':
            best_run = dict_to_df[dict_to_df[1] == dict_to_df[1].min()].index.values
            best_run = best_run[0]

        model = sm.tsa.statespace.SARIMAX(train,
                                          order=myDict[best_run][2],
                                          seasonal_order=myDict[best_run][3],
                                          enforce_stationarity=False,
                                          enforce_invertibility=False).fit()

        best_model = {'model':model, 
                      'aic':model.aic,
                      'bic':model.bic,
                      'order':myDict[best_run][2], 
                      'seasonal_order':myDict[best_run][3]}

        return best_model

    best = find_best_sarima(train, 'aic')
    #pred = best['model'].predict(start=test.index[0], end=test.index[-1], dynamic=True)
    predx = best['model'].forecast(len(test))
    error = mean_squared_error(test, predx, squared = False)
    #r21 = r2_score(test, predx)
    print('SARIMA Test RMSE: %.3f' % error)
    #print('SARIMA Test Test rˆ2: %.3f' % r21)
    sarimaErrors.append(error) 
    #debug:
    print("--------------DEBUG-------------\n")
    for pair in best.items():
      print(pair)
    return best

In [ ]:
def UnobservedComponents(train, test):
    # smooth trend model without seasonal or cyclical components
    model = {
        'level': 'smooth trend', 'cycle': False, 'seasonal': None, 
    }
    # https://www.statsmodels.org/dev/generated/statsmodels.tsa.statespace.structural.UnobservedComponents.html
    gcag_mod = sm.tsa.UnobservedComponents(train['X'], **model)
    gcag_res = gcag_mod.fit()
    
    seasonal_model = {
        'level': 'local linear trend',
        'seasonal': 12
    }
    mod = sm.tsa.UnobservedComponents(train['X'], **seasonal_model)
    res = mod.fit(method='powell', disp=False)
    
    seasonal_model2 = {
    'level': 'local level',
    'seasonal': 12
    }
    llmod = sm.tsa.UnobservedComponents(train['X'], **seasonal_model2)
    ll_level_res = llmod.fit(method='powell', disp=False)
    
    # smooth trend model without seasonal components with cycle component
    cyclemodel = {
        'level': 'smooth trend', 'cycle': True, 'seasonal': None, 
    }
    # https://www.statsmodels.org/dev/generated/statsmodels.tsa.statespace.structural.UnobservedComponents.html
    cycle_mod = sm.tsa.UnobservedComponents(train['X'], **cyclemodel)
    cycle_res = cycle_mod.fit()
    '''
    seasonal_model365 = {
        'level': 'local linear trend',
        'seasonal': 365
    }
    mod365 = sm.tsa.UnobservedComponents(train['X'], **seasonal_model365)
    res365 = mod365.fit(method='powell', disp=False)
    '''
    predx2 = gcag_res.forecast(len(test))
    error2 = mean_squared_error(test, predx2, squared = False)
    #r22 = r2_score(test, predx2)
    print('UnobsComp. Test RMSE: %.3f' % error2)
    #print('UnobsComp. Test rˆ2: %.3f' % r22)
    predx3 = cycle_res.forecast(len(test))
    error3 = mean_squared_error(test, predx3, squared = False)
    #r23 = r2_score(test, predx3)
    print('UnobsComp. Cyclical Test RMSE: %.3f' % error3)
    #print('UnobsComp. Test rˆ2: %.3f' % r23)
    predx4 = res.forecast(len(test))
    error4 = mean_squared_error(test, predx4, squared = False)
    #r24 = r2_score(test, predx4)
    print('UnobsComp. Seasonal Test RMSE: %.3f' % error4)
    #print('UnobsComp. Test rˆ2: %.3f' % r24)
    predx5 = ll_level_res.forecast(len(test))
    error5 = mean_squared_error(test, predx5, squared = False)
    #r25 = r2_score(test, predx5)
    print('UnobsComp. Seasonal Local Level Test RMSE: %.3f' % error5)
    #print('UnobsComp. Test rˆ2: %.3f' % r25)
    '''
    predx6 = res365.forecast(len(test))
    error6 = mean_squared_error(test, predx6, squared = False)
    #r26 = r2_score(test, predx6)
    print('UnobsComp. 365 Seasonal Test RMSE: %.3f' % error6)
    #print('UnobsComp. Test rˆ2: %.3f' % r26)
    '''

Results

In [ ]:
#driverResults = pd.DataFrame()
for x in range(6):
    print("Iteration # "+str(x))
   # driverResults[str(x)] = 0
    y = str(2013+x)
    z = str(2014+x)
    print("Current Training set is 2012 : "+y)
   # driverResults.iat[0,x]= "Current Training set is 2012 : "+str(y)
    print("Current Testing set is "+z+" : 2019-12")
   # driverResults.iat[1,x]= "Current Testing set is "+str(z)+" : 2019-12"
    #Rolling Mean 
    meanTrain = df2['2012':y]
    meanTest = df2[z:'2019-12']
    #Rolling Median
    medianTrain = df4['2012':y]
    medianTest = df4[z:'2019-12']
    #0s
    zeroTrain = df1['2012':y]
    zeroTest = df1[z:'2019-12']
    #Driver
    print("Rolling Mean")
    #driverResults.iat[2,x]="Rolling Mean"
    SARIMA(meanTrain,meanTest)
   # driverResults.iat[3,x]="SARIMA"
    UnobservedComponents(meanTrain,meanTest)
    #driverResults.iat[3,x]="UnobservedComponents"
    print("Rolling Median")
    SARIMA(medianTrain,medianTest)
    UnobservedComponents(medianTrain,medianTest)
    print("0s")
    SARIMA(zeroTrain,zeroTest)
    UnobservedComponents(zeroTrain,zeroTest)  

Iteration # 0
Current Training set is 2012 : 2013
Current Testing set is 2014 : 2019-12
Rolling Mean
SARIMA Test RMSE: 1.078
--------------DEBUG-------------

('model', <statsmodels.tsa.statespace.sarimax.SARIMAXResultsWrapper object at 0x7efdcaad2320>)
('aic', 2148.2685814325405)
('bic', 2166.5013726437896)
('order', (1, 0, 1))
('seasonal_order', (0, 1, 1, 12))
UnobsComp. Test RMSE: 1.479
UnobsComp. Cyclical Test RMSE: 1.556
UnobsComp. Seasonal Test RMSE: 1.388
UnobsComp. Seasonal Local Level Test RMSE: 1.093
Rolling Median
SARIMA Test RMSE: 1.097
--------------DEBUG-------------

('model', <statsmodels.tsa.statespace.sarimax.SARIMAXResultsWrapper object at 0x7efdcaad25c0>)
('aic', 2171.463519253656)
('bic', 2189.6963104649053)
('order', (1, 0, 1))
('seasonal_order', (0, 1, 1, 12))
UnobsComp. Test RMSE: 1.288
UnobsComp. Cyclical Test RMSE: 1.293
UnobsComp. Seasonal Test RMSE: 1.399
UnobsComp. Seasonal Local Level Test RMSE: 1.113
0s
SARIMA Test RMSE: 1.340
--------------DEBUG---------

#FB Prophet

In [ ]:
!pip install fbprophet

In [ ]:
from fbprophet import Prophet

In [ ]:
df2test = df2['2019':'2019-12']
df2train = df2['2012':'2018']

In [ ]:
# New data
fb_data = df2train.reset_index()
fb_data.rename(columns={"index":"ds", "X":"y"}, inplace=True)

## Additive Model

In [ ]:
my_model = Prophet(interval_width=0.95)

In [ ]:
my_model.fit(fb_data)

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [ ]:
future = my_model.make_future_dataframe(periods=365)


In [ ]:
fbpredx4 = my_model.predict(future)


In [ ]:
fbforecat = fbpredx4.yhat[-365:]

In [ ]:
fberror4 = mean_squared_error(df2test, fbforecat, squared = False)
print('FB Prophet Test RMSE: %.3f' % fberror4)

FB Prophet Test RMSE: 0.977


## Multiplicative Model

In [ ]:
my_model_multiplicative = Prophet(interval_width=0.95, seasonality_mode='multiplicative')

In [ ]:
my_model_multiplicative.fit(fb_data)

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [ ]:
#future_dates = my_model_multiplicative.make_future_dataframe(periods=36, freq='MS')
future_dates = my_model_multiplicative.make_future_dataframe(periods=365)

In [ ]:
fbmpredx4 = my_model_multiplicative.predict(future_dates)


In [ ]:
fbmforecat = fbmpredx4.yhat[-365:]

In [ ]:
fbmerror4 = mean_squared_error(df2test, fbmforecat, squared = False)
print('FB Prophet Test RMSE: %.3f' % fbmerror4)

FB Prophet Test RMSE: 0.976
